In [1]:
# Change to project's root directory
%cd "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard"

/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard


In [2]:
import os
import psycopg2
import pandas as pd
from google.colab import userdata

# Get credentials from Colab secrets.
db_user = userdata.get('DB_USER')
db_pass = userdata.get('DB_PASS')
db_host = userdata.get('DB_HOST')
db_port = userdata.get('DB_PORT')
db_name = userdata.get('DB_NAME')

In [3]:
!pip install -q psycopg2-binary
!pip install -q dbt-postgres --use-deprecated=legacy-resolver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.9/985.9 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.7/442.7 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Install PostgreSQL
!apt-get -y -qq install postgresql-14 >/dev/null

# Start the PostgreSQL server
!sudo service postgresql start

# Create the new database user and set its password
!sudo -u postgres psql -c "CREATE USER {db_user} WITH PASSWORD '{db_pass}';"

# Create the database and grant permissions
!sudo -u postgres createdb financial_data
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE financial_data TO {db_user};"

print("PostgreSQL setup complete. Database is ready for connection.")

 * Starting PostgreSQL 14 database server
   ...done.
could not change directory to "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard": Permission denied
CREATE ROLE
could not change directory to "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard": Permission denied
could not change directory to "/content/drive/MyDrive/Colab Notebooks/FinancialForecastingDashboard": Permission denied
GRANT
PostgreSQL setup complete. Database is ready for connection.


In [5]:
try:
    # Establish a connection to the database
    conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        dbname=db_name,
        user=db_user,
        password=db_pass
    )
    print("Database connection successful.")

except psycopg2.OperationalError as e:
    print(f"Connection error: {e}")
    conn = None

Database connection successful.


In [6]:
# Ingest raw data and create the raw_saas_metrics table
!DB_HOST={db_host} DB_PORT={db_port} DB_NAME={db_name} DB_USER={db_user} DB_PASS={db_pass} python "scripts/database_setup.py"

Successfully connected to the database.
Table 'raw_saas_metrics' is ready.
Successfully ingested 5000 records into raw_saas_metrics.
Database connection closed.


In [7]:
# Run the dbt pipeline.
!DB_HOST={db_host} DB_PORT={db_port} DB_NAME={db_name} DB_USER={db_user} DB_PASS={db_pass} python "scripts/pipeline.py"

profiles.yml created successfully.
Creating core dbt project files...
Running the dbt pipeline...
09:12:04  Running with dbt=1.10.13
09:12:04  Registered adapter: postgres=1.9.1
09:12:07  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.financial_forecasting.example
09:12:09  Found 2 models, 3 data tests, 1 source, 447 macros
09:12:09  
09:12:09  Concurrency: 1 threads (target='dev')
09:12:09  
09:12:09  1 of 2 START sql view model public.stg_raw_saas_metrics_data ................... [RUN]
09:12:11  1 of 2 OK created sql view model public.stg_raw_saas_metrics_data .............. [CREATE VIEW in 2.44s]
09:12:11  2 of 2 START sql view model public.fact_monthly_revenue ........................ [RUN]
09:12:12  2 of 2 OK created sql view model public.fact_monthly_revenue ................... [CREATE VIEW in 1.23s]
09:12:12  
09:12:12  Finished running 2 view models in 0 hours 0 minutes and 

In [8]:
if conn:
    try:
        # SQL query to select the two columns needed for forecasting
        query = "SELECT subscription_month, monthly_recurring_revenue FROM public.fact_monthly_revenue ORDER BY subscription_month;"

        # Use pandas to read the SQL query into a DataFrame.
        revenue_df = pd.read_sql(query, conn)

        # Print the first few rows and the info to confirm it worked
        print(revenue_df.head())
        print(revenue_df.info())

    except Exception as e:
        print(f"An error occurred while fetching data: {e}")

    finally:
        # Close the connection
        if conn:
            conn.close()
            print("Database connection closed.")

         subscription_month  monthly_recurring_revenue
0 2022-01-01 00:00:00+00:00              341683.994540
1 2022-02-01 00:00:00+00:00              338381.395471
2 2022-03-01 00:00:00+00:00              400742.214030
3 2022-04-01 00:00:00+00:00              348733.482907
4 2022-05-01 00:00:00+00:00              353009.560476
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   subscription_month         36 non-null     datetime64[ns, UTC]
 1   monthly_recurring_revenue  36 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 708.0 bytes
None
Database connection closed.


/tmp/ipython-input-1123113102.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  revenue_df = pd.read_sql(query, conn)


--- Ran the pipeline and set up df. Now to create and train the forecasting model ---

In [9]:
# Making sure prophet is installed
!pip install -q prophet

In [10]:
from prophet import Prophet

# Prepare Data for Prophet
# Rename columns to Prophet's required 'ds' (datestamp) and 'y' (value)
prophet_df = revenue_df.rename(columns={
    'subscription_month': 'ds',
    'monthly_recurring_revenue': 'y'
})

# Ensure 'ds' is a datetime object— crucial for Prophet
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

# Drop the timezone information, Prophet only accepts naive datetime objects.
if prophet_df['ds'].dt.tz is not None:
    prophet_df['ds'] = prophet_df['ds'].dt.tz_localize(None)

# Initialize and Train the Model
# Configure Prophet:
# - yearly_seasonality=True: Since revenue often has seasonal peaks (like year-end sales, etc).
# - changepoint_prior_scale=0.05: Controls how flexible the trend is. 0.05 is a good medium starting point for a stable financial series.
m = Prophet(
    yearly_seasonality=True,
    changepoint_prior_scale=0.05
)

# Fit the model.
m.fit(prophet_df)

print("Prophet model training complete.")

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfi51tre6/b1nvi3dd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfi51tre6/tn3yk7nm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3145', 'data', 'file=/tmp/tmpfi51tre6/b1nvi3dd.json', 'init=/tmp/tmpfi51tre6/tn3yk7nm.json', 'output', 'file=/tmp/tmpfi51tre6/prophet_modelwnc6_in9/prophet_model-20251001091459.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
09:14:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:14:59 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet model training complete.


--- Generate time-series forecast for 24 months ---

In [11]:
# Creating a dataFrame with 24 future 'periods' at a frequency (months, 'MS'='month start')
future = m.make_future_dataframe(periods=24, freq='MS')
print(future.tail())

# Generate the forecast for future dates.
# Output here is a df containing predictions and uncertainty intervals.
forecast = m.predict(future)

print("\nForecast generated successfully. DataFrame columns:")
print(forecast.columns)

           ds
55 2026-08-01
56 2026-09-01
57 2026-10-01
58 2026-11-01
59 2026-12-01

Forecast generated successfully. DataFrame columns:
Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
       'yearly', 'yearly_lower', 'yearly_upper', 'multiplicative_terms',
       'multiplicative_terms_lower', 'multiplicative_terms_upper', 'yhat'],
      dtype='object')


--- Clean the forecasted data and push to database ---

In [12]:
from sqlalchemy import create_engine

# Clean the forecast dataframe.

# Keep only the relevant columns in a new df.
final_forecast_df = forecast[[
    'ds',            # The date
    'yhat',          # The point prediction (forecasted MRR)
    'yhat_lower',    # The lower bound of the prediction interval
    'yhat_upper'     # The upper bound of the prediction interval
]].copy()

# Rename the month and prediction columns for clarity in the database
final_forecast_df = final_forecast_df.rename(columns={'yhat': 'forecasted_mrr', 'ds': 'subscription_month'})

print("Forecast DataFrame cleaned and ready for ingestion.")
print(final_forecast_df.tail())


Forecast DataFrame cleaned and ready for ingestion.
   subscription_month  forecasted_mrr     yhat_lower     yhat_upper
55         2026-08-01   336810.642202  305517.824208  365591.841488
56         2026-09-01   336666.968282  307301.709088  362292.871389
57         2026-10-01   379952.174536  349614.756758  410489.490198
58         2026-11-01   370279.906376  343384.499341  399985.697669
59         2026-12-01   329600.829504  299612.139123  357613.444465


In [13]:
# Ingest into PostgreSQL

# Create a SQLAlchemy engine for bulk insertion
# We set the connection variables at start of notebook
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

# Write the DataFrame to a new table.
# if_exists='replace': Recreates the table every time.
# index=False: Don't write the pandas index as a column.
final_forecast_df.to_sql(
    'fact_monthly_revenue_forecast',  # new table name
    engine,
    if_exists='replace',
    index=False
)

print("\nForecast data successfully loaded into 'fact_monthly_revenue_forecast' table.")


Forecast data successfully loaded into 'fact_monthly_revenue_forecast' table.
